# Train model

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import torch

from load_qm9 import *
from display_mol import *

#load dataset
ds = qm9_load_tfdata()

## data management and preprocessing

### Moldecule dataloader for trainning in batchs of equal size

In [2]:
ATOM_TYPES = [1, 6, 7, 8, 9]
MAX_SIZE = 30 # max number of atoms per molecules


class MoleculeDataset:
    def __init__(self, tf_dataset):
        self.ds = tf_dataset
        self.molecules_list = list(self.ds.as_numpy_iterator()) # Convertir le dataset en liste pour pouvoir indexer
        self.N = tf_dataset.reduce(0, lambda x, _: x + 1).numpy()
        self.M_pdf = np.zeros(MAX_SIZE, dtype=np.int32)
        self.molecules_by_size = {i: [] for i in range(MAX_SIZE)}
        
        self._index_molecules()
        
        print(f"Dataset chargé : {self.N} molécules")
        print(f"Distribution des tailles : {self.M_pdf}")
    

    def _index_molecules(self):
        for idx, example in enumerate(self.molecules_list):
            n_atoms = example[1].shape[0]
            self.M_pdf[n_atoms] += 1
            self.molecules_by_size[n_atoms].append(idx)
    
    
    def get_batch(self, batch_size): 
        """
        return generator that give infinite patchs with molecules of equal sizes
        """ 
        valid_sizes = [M for M in range(MAX_SIZE) if self.M_pdf[M] >= batch_size]

        valid_probs = np.array([self.M_pdf[M] for M in valid_sizes])
        valid_probs = valid_probs / valid_probs.sum() 
        
        while True:
            M = np.random.choice(valid_sizes, p=valid_probs)
            
            available_indices = self.molecules_by_size[M]
            
            selected_indices = np.random.choice( # shuffle
                    available_indices, 
                    size=batch_size, 
                    replace=False
                )
            
            batch = [self.molecules_list[idx] for idx in selected_indices]
            
            yield self._collate_batch(batch)
    
    def _collate_batch(self, batch):
        """
        Combine une liste de molécules en un batch.
        """
        x = np.stack([mol[0] for mol in batch], axis=0)
        y = np.stack([mol[1] for mol in batch], axis=0)
        q = np.stack([mol[2] for mol in batch], axis=0)
        
        return [x, y, q]
    
    def get_epoch_batches(self, batch_size):
        """
        Génère des batches pour une époque complète.
        Chaque molécule apparaît exactement une fois par époque.
        """
        all_batches = []
        
        for M in range(MAX_SIZE):
            if self.M_pdf[M] == 0:
                continue
            
            indices = self.molecules_by_size[M].copy()
            
            np.random.shuffle(indices)
            
            for i in range(0, len(indices), batch_size):
                batch_indices = indices[i:i + batch_size]
                if len(batch_indices) == batch_size:
                    batch = [self.molecules_list[idx] for idx in batch_indices]
                    all_batches.append(batch)
        
        np.random.shuffle(all_batches)
        
        for batch in all_batches:
            yield self._collate_batch(batch)
    

molecules = MoleculeDataset(ds) # ~1min30


Dataset chargé : 133885 molécules
Distribution des tailles : [    0     0     0     2     4     5    12    21    70   193   527  1150
  2336  4259  7103 10646 14270 17394 17836 18336 12601 13189  4483  6362
   713  1923    59   356     0    35]


In [16]:


# Génération infinie de batches (pour l'entraînement)
batch_gen = molecules.get_batch(batch_size=32)

for i in range(5):
    batch = next(batch_gen)
    print([i.shape for i in batch])

"""
# Méthode 2 : Une epoch complète (chaque molécule vue exactement une fois)
print("\epoch complète:")
for epoch in range(2):
    print(f"\nÉpoque {epoch + 1}")
    batch_count = 0
    for batch in molecules.get_epoch_batches(batch_size=32, shuffle=True):
        batch_count += 1
        if batch_count <= 3:  # Afficher les 3 premiers batches
            print(f"  Batch {batch_count}: {batch['N'][0]} atomes, {len(batch['N'])} molécules")
    print(f"  Total: {batch_count} batches")"""

[(32, 9, 3), (32, 9), (32, 9)]
[(32, 17, 3), (32, 17), (32, 17)]
[(32, 16, 3), (32, 16), (32, 16)]
[(32, 17, 3), (32, 17), (32, 17)]
[(32, 21, 3), (32, 21), (32, 21)]


'\n# Méthode 2 : Une epoch complète (chaque molécule vue exactement une fois)\nprint("\\epoch complète:")\nfor epoch in range(2):\n    print(f"\nÉpoque {epoch + 1}")\n    batch_count = 0\n    for batch in molecules.get_epoch_batches(batch_size=32, shuffle=True):\n        batch_count += 1\n        if batch_count <= 3:  # Afficher les 3 premiers batches\n            print(f"  Batch {batch_count}: {batch[\'N\'][0]} atomes, {len(batch[\'N\'])} molécules")\n    print(f"  Total: {batch_count} batches")'

### one hot encoding for the molecule shape

In [46]:
def one_hot_encode(batch):
    x = torch.tensor(batch[0], dtype=torch.float32)
    e = torch.tensor(batch[1], dtype=torch.long)
    q = torch.tensor(batch[2], dtype=torch.float32)
    
    max_atom_type = max(ATOM_TYPES) + 1
    lookup = torch.full((max_atom_type,), -1, dtype=torch.long)
    
    for idx, atom_type in enumerate(ATOM_TYPES):
        lookup[atom_type] = idx
    
    y_indices = lookup[e]
    
    assert (y_indices >= 0).all(), "Types d'atomes inconnus détectés!"
    
    
    e = torch.nn.functional.one_hot(y_indices, num_classes=len(ATOM_TYPES)).float()
    
    return [x, e, q]

def one_hot_decode(batch):
    x = batch[0].clone().detach()
    e_onehot = batch[1].clone().detach()
    q = batch[2].clone().detach()
    
    e_indices = torch.argmax(e_onehot, dim=-1)  # (batch_size, n_atoms)
    atom_types_tensor = torch.tensor(ATOM_TYPES, dtype=torch.long)
    e = atom_types_tensor[e_indices]
    
    return [x, e, q]



def one_hot_decode_stochastic(batch, temperature=1.0):
    x = batch[0].clone().detach()
    e_soft = batch[1].clone().detach()
    q = batch[2].clone().detach()
    
    # Option 2: Sampling selon les probabilités (stochastic)
    e_probs = torch.nn.functional.softmax(e_soft / temperature, dim=-1)
    e_indices = torch.multinomial(
        e_probs.view(-1, e_probs.size(-1)), 
        num_samples=1
    ).view(e_probs.shape[:-1])
    
    atom_types_tensor = torch.tensor(ATOM_TYPES, dtype=torch.long)
    e = atom_types_tensor[e_indices]
    
    return [x, e, q]
    

mol = one_hot_encode(next(batch_gen))   
print([e.shape for e in mol])
print([e.shape for e in one_hot_decode(mol)])

[torch.Size([32, 18, 3]), torch.Size([32, 18, 5]), torch.Size([32, 18])]
[torch.Size([32, 18, 3]), torch.Size([32, 18]), torch.Size([32, 18])]


## Model

In [65]:
class UltraMagaDiffusion(torch.nn.Module):
    def __init__(self):
        super(UltraMagaDiffusion, self).__init__()
        self.T = 10
        s = 1e-5
        self.alpha = [(1-2*s)*(1 - (t/self.T))+s for t in range(self.T+1)]
        self.omega = [1-alpha**2 for alpha in self.alpha]
        self.L = 9
        self.lr = 1e-4
        self.num_class = len(ATOM_TYPES)

        self.init_weight()
    
    def init_weight(self):
        self.phi_e = torch.nn.ModuleList()
        self.phi_inf = torch.nn.ModuleList()
        self.phi_x = torch.nn.ModuleList()
        self.phi_h = torch.nn.ModuleList()

        nf = self.num_class + 2  # +2 for atom charge and t/T

        for l in range(self.L):
            self.phi_e.append(torch.nn.Sequential(
                torch.nn.Linear(nf * 2 + 2, nf), 
                torch.nn.SiLU(),
                torch.nn.Linear(nf, nf),
                torch.nn.SiLU(),
            ))

            self.phi_inf.append(torch.nn.Sequential(
                torch.nn.Linear(nf, 1),
                torch.nn.Sigmoid(),
            ))

            self.phi_x.append(torch.nn.Sequential(
                torch.nn.Linear(nf * 2 + 2, nf),
                torch.nn.SiLU(),
                torch.nn.Linear(nf, nf),
                torch.nn.SiLU(),
                torch.nn.Linear(nf, 1),
            ))

            self.phi_h.append(torch.nn.Sequential(
                torch.nn.Linear(nf * 2, nf),
                torch.nn.SiLU(),
                torch.nn.Linear(nf, nf),
            ))
        
    
    def forward(self, mols, t):
        (x, e, q) = mols 
        h = torch.cat([e, q[:, :, None], torch.ones_like(q)[:, :, None]*t[:, :, None]/self.T], dim=-1)
        x0 = x.clone()
        N = h.shape[1]
        diff = x[:, :, None, :] - x[:, None, :, :]   # (B, N, N, 3)
        a_ij = torch.sqrt(torch.sum(diff ** 2, dim=-1))[:, :, :, None]  # (B, N, N, 1)
        for l in range(self.L):
            diff = x[:, :, None, :] - x[:, None, :, :]   # (B, N, N, 3)
            d_ij = torch.sqrt(torch.sum(diff ** 2, dim=-1))[:, :, :, None]  # (B, N, N, 1) distance squared matrix

            # compute m_ij
            h_i = h[:, :, None, :].expand(-1, N, N, -1)  # (B, N, N, d)
            h_j = h[:, None, :, :].expand(-1, N, N, -1)  # (B, N, N, d)

            features = torch.cat([h_i, h_j, d_ij, a_ij], dim=-1)  # (B, N, N, 2d+2)
            m_ij = self.phi_e[l](features)  # (B, N, N, d)

            # compute e_ij
            e_ij = self.phi_inf[l](m_ij) # (B, N, N, 1)

            # update x
            weights = self.phi_x[l](features) * diff / (d_ij + 1.0)  # (B, N, N, 3)
            x = x + weights.sum(dim=2)                          # (B, N, 3)

            # update h
            mask = ~torch.eye(N, dtype=bool)[None, :, :, None]  # (1, N, N, 1)
            agg = (e_ij * mask * m_ij).sum(dim=2)   # (B, N, d)
            h = h + self.phi_h[l](torch.cat([h, agg], dim=-1))   # (B, N, d)

    
        x = x - x0 # (B, N, 3)
        x = x - torch.mean(x, axis=1)[:, None, :] # (B, 1, 3)
        e, q = h[:, :, :self.num_class], h[:, :, self.num_class]
        return (x, e, q)
    

    def training_step(self, batch, optimizer):
        self.train()
        optimizer.zero_grad()

        x, e, q = batch
        B = x.shape[0]

        t = torch.randint(1, self.T + 1, (B, 1)).float() # Random timestep per batch
        eps_x = torch.randn_like(x) # (B, N, 3)
        eps_e = torch.randn_like(e) # (B, N, D)
        eps_q = torch.randn_like(q) # (B, N)

        # Diffusion step: add noise to x0
        alpha_t = torch.tensor(self.alpha)[t.long()].view(B, 1, 1)
        omega_t = torch.sqrt(torch.tensor(self.omega))[t.long()].view(B, 1, 1)
        x_t = alpha_t * x + omega_t * eps_x
        e_t = alpha_t * e + omega_t * eps_e
        q_t = alpha_t.squeeze(-1) * q + omega_t.squeeze(-1) * eps_q

        # Predict denoised offset (model learns to predict ε)
        eps_x_pred, eps_e_pred, eps_q_pred = self.forward((x_t, e_t, q_t), t)

        # Reconstruction loss (denoising loss)
        loss_x = torch.nn.functional.mse_loss(eps_x_pred, eps_x)
        loss_e = torch.nn.functional.mse_loss(eps_e_pred, eps_e)
        loss_q = torch.nn.functional.mse_loss(eps_q_pred, eps_q)

        loss = loss_x + 0.25 * loss_e + 0.1 * loss_q # coef from the paper
        loss.backward()
        optimizer.step()

        return {
            'loss': loss.item(),
            'loss_x': loss_x.item(),
            'loss_e': loss_e.item(),
            'loss_q': loss_q.item(),
        }


    @torch.no_grad()
    def sample(self, batch_size, N):
        self.eval()

        # Initialize x_T from Gaussian noise
        x = torch.randn(batch_size, N, 3)
        e = torch.randn(batch_size, N, self.num_class)
        q = torch.randn(batch_size, N)

        for t in reversed(range(1, self.T + 1)):
            t_tensor = torch.full((batch_size, 1), t).float()
            dx, de, dq = self.forward((x, e, q), t_tensor)

            # Compute coefficients
            alpha_t = torch.tensor(self.alpha)[t].view(1, 1, 1)
            omega_t = torch.tensor(self.omega)[t].view(1, 1, 1)
            alpha_s = torch.tensor(self.alpha)[t-1].view(1, 1, 1)
            omega_s = torch.tensor(self.omega)[t-1].view(1, 1, 1)

            alpha_ts = alpha_t/alpha_s
            omega_ts = omega_t - alpha_ts**2 * omega_s
            omega_t_s = torch.sqrt(omega_ts * omega_s / omega_t)
            # Reverse diffusion step
            eps_x = torch.randn_like(x) if t > 1 else 0.0
            eps_e = torch.randn_like(e) if t > 1 else 0.0
            eps_q = torch.randn_like(q) if t > 1 else 0.0
            x = x / alpha_s - omega_ts/(alpha_ts*torch.sqrt(omega_t)) * dx + omega_t_s * eps_x
            e = e / alpha_s - omega_ts/(alpha_ts*torch.sqrt(omega_t)) * de + omega_t_s * eps_e
            q = q / alpha_s.unsqueeze(-1) - (omega_ts/(alpha_ts*torch.sqrt(omega_t))).unsqueeze(-1) * dq + omega_t_s.unsqueeze(-1) * eps_q

        return x, e, q




model = UltraMagaDiffusion()
optimizer = torch.optim.Adam(model.parameters(), lr=model.lr)
batch_gen = molecules.get_batch(batch_size=32)

for i in range(10):
    batch = one_hot_encode(next(batch_gen))
    logs = model.training_step(batch, optimizer)
    print(logs)

# Sampling
#x_gen, e_gen, q_gen = model.sample(batch_size=8, N=9)


{'loss': 4.76224422454834, 'loss_x': 4.501723289489746, 'loss_e': 0.6904703378677368, 'loss_q': 0.8790343403816223}
{'loss': nan, 'loss_x': nan, 'loss_e': nan, 'loss_q': nan}
{'loss': nan, 'loss_x': nan, 'loss_e': nan, 'loss_q': nan}
{'loss': nan, 'loss_x': nan, 'loss_e': nan, 'loss_q': nan}
{'loss': nan, 'loss_x': nan, 'loss_e': nan, 'loss_q': nan}
{'loss': nan, 'loss_x': nan, 'loss_e': nan, 'loss_q': nan}
{'loss': nan, 'loss_x': nan, 'loss_e': nan, 'loss_q': nan}
{'loss': nan, 'loss_x': nan, 'loss_e': nan, 'loss_q': nan}
{'loss': nan, 'loss_x': nan, 'loss_e': nan, 'loss_q': nan}
{'loss': nan, 'loss_x': nan, 'loss_e': nan, 'loss_q': nan}
